In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import torch
torch.set_printoptions(precision=10)

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from models.ranked_transformer import HsqcRankedTransformer

from models.optional_input_ranked_transformer import OptionalInputRankedTransformer
from datasets.oneD_dataset import OneDDataModule

from datasets.optional_2d_folder_dataset import OptionalInputDataModule
import yaml
from pytorch_lightning.loggers import TensorBoardLogger
torch.set_float32_matmul_precision('medium')

In [11]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")
warnings.filterwarnings("ignore", category=UserWarning, message="The PyTorch API of nested tensors is in prototype stage and will change in the near future.")


In [3]:
from datasets.dataset_utils import  fp_loader_configer
fp_loader_configer.select_version("MFP_Specific_Radius")
fp_loader = fp_loader_configer.fp_loader


Here we compare models' performance over 7 kinds of inputs


In [36]:
# load model 

checkpoint_path = Path("/root/gurusmart/MorganFP_prediction/reproduce_previous_works/stable_argsort/flexible_models_best_FP/r0_r2_FP_trial_1/checkpoints/epoch=32-all_inputs.ckpt")
model_path = checkpoint_path.parents[1]


hyperpaerameters_path = model_path / "hparams.yaml"

with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
    
FP_building_type = hparams['FP_building_type'].split("_")[-1]
only_2d = not hparams['use_oneD_NMR_no_solvent']
fp_loader.setup(only_2d=only_2d,FP_building_type=FP_building_type)
fp_loader.set_max_radius(int(hparams['FP_choice'].split("_")[-1][1:]), only_2d=only_2d)


del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
# hparams['use_peak_values'] = False




loading time: 124.77609610557556
counting time: 10.722865104675293
finish entropy list


In [37]:
model = HsqcRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)
# model = HsqcRankedTransformer(**hparams)
model.change_ranker_for_testing()
# model.change_ranker_for_testing(test_ranking_set_path = "/workspace/ranking_sets_cleaned_by_inchi/SMILES_R0_to_R4_reduced_FP_ranking_sets_only_all_info_molecules/test/rankingset.pt")
# model=model.to("cuda")
model.device

Initialized SignCoordinateEncoder[384] with dims [180, 180, 24] and 2 positional encoders. 24 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args


device(type='cuda', index=0)

In [38]:
datamodule = OptionalInputDataModule(dir="/workspace/SMILES_dataset", FP_choice=hparams["FP_choice"], input_src=["HSQC", "oneD_NMR"], batch_size=hparams['bs'], parser_args=hparams)
datamodule.setup("test")
loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR = \
    datamodule.test_dataloader()
    
    
# datamodule = OneDDataModule(dir="/workspace/SMILES_dataset", FP_choice=hparams["FP_choice"], batch_size=hparams['bs'], parser_args=hparams) 
# datamodule.setup("test")
# loader_only_C_NMR = datamodule.test_dataloader()
    


In [39]:
trainer = Trainer()
metric_to_focus = [
    'rank_1',
    'rank_5',
    "mean_rank",
    'cos',
    'f1'    
]
def show_model_performance(model, out_file ="temp_model_show.txt"):
    # loaders = [loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR]
    # names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
    # loaders = [ loader_only_1d]
    # names = [ "only_1d"]
    loaders = [loader_all_inputs]
    names = ["all_inputs"]
    # names = ["only_hsqc"]
    with open(out_file, "w") as file:
        for loader, name in zip(loaders, names):
            result =  trainer.test(model, dataloaders=loader) 
            # result = trainer.test(model, datamodule, ckpt_path = checkpoint_path)
            file.write(f"\n{name:10}: ")
            for m in metric_to_focus:
                file.write(f"{m}: {result[0]['test/mean_'+m]:.4f}, ")
    return result

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [40]:
result = show_model_performance(model)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                      Test metric                       ┃                      DataLoader 0                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                   test/mean_accuracy                   │                   0.9969945549964905                   │
│                 test/mean_active_bits                  │                   97.28133392333984                    │
│                   test/mean_ce_loss                    │                  0.011952337808907032                  │
│                     test/mean_cos                      │                   0.9030303955078125                   │
│                      test/mean_f1                      │                   0.9077176451683044                   │
│                   test/mean_jaccard                    │                   0.8319511413574219                   │
│                  test/mean_mean_rank                   │                      1.388671875                       │
│                   test/mean_neg_loss                   │                  0.003288800595328212                  │
│                   test/mean_pos_loss                   │                  0.008663536980748177                  │
│                 test/mean_pos_neg_loss                 │                  0.011952337808907032                  │
│                  test/mean_precision                   │                   0.9420560002326965                   │
│                    test/mean_rank_1                    │                      0.947265625                       │
│                   test/mean_rank_10                    │                     0.994873046875                     │
│                    test/mean_rank_5                    │                     0.99365234375                      │
│                    test/mean_recall                    │                   0.8759701251983643                   │
│        test/rank_1_of_NP_class/Alkylresorcinols        │                          1.0                           │
│     test/rank_1_of_NP_class/Amino acid glycosides      │                          1.0                           │
│        test/rank_1_of_NP_class/Aminosugars and         │                          1.0                           │
│                    aminoglycosides                     │                                                        │
│   test/rank_1_of_NP_class/Anthranilic acid alkaloids   │                         0.9375                         │
│         test/rank_1_of_NP_class/Apocarotenoids         │                          1.0                           │
│      test/rank_1_of_NP_class/Aromatic polyketides      │                   0.9215686321258545                   │
│       test/rank_1_of_NP_class/Carotenoids (C40)        │                          1.0                           │
│       test/rank_1_of_NP_class/Carotenoids (C50)        │                          1.0                           │
│           test/rank_1_of_NP_class/Chromanes            │                          1.0                           │
│           test/rank_1_of_NP_class/Coumarins            │                   0.9666666388511658                   │
│       test/rank_1_of_NP_class/Cyclic polyketides       │                   0.9622641801834106                   │
│        test/rank_1_of_NP_class/Diarylheptanoids        │                          1.0                           │
│    test/rank_1_of_NP_class/Diazotetronic acids and     │                   0.800000011920929                    │
│                      derivatives                       │                                                        │
│     test/rank_1_of_NP_class/Diphenyl ethers (DPEs)     │                          1.0                           │
│          test/rank_1_of_NP_class/Diterpenoids         

In [35]:
result

[{'test/mean_ce_loss': 0.015217501670122147,
  'test/mean_pos_loss': 0.011341625824570656,
  'test/mean_neg_loss': 0.003875875845551491,
  'test/mean_pos_neg_loss': 0.015217501670122147,
  'test/mean_cos': 0.9062198400497437,
  'test/mean_jaccard': 0.8406772017478943,
  'test/mean_active_bits': 100.24747467041016,
  'test/mean_f1': 0.9135006070137024,
  'test/mean_precision': 0.9334216117858887,
  'test/mean_recall': 0.8945155739784241,
  'test/mean_accuracy': 0.9971511960029602,
  'test/mean_mean_rank': 0.79052734375,
  'test/mean_rank_1': 0.9566243290901184,
  'test/mean_rank_5': 0.9890950322151184,
  'test/mean_rank_10': 0.9917805790901184,
  'test/rank_1_of_NP_class/Saccharides': 0.8999999761581421,
  'test/rank_1_of_NP_class/Diterpenoids': 0.9890829920768738,
  'test/rank_1_of_NP_class/Monoterpenoids': 0.9469026327133179,
  'test/rank_1_of_NP_class/Sesquiterpenoids': 0.9837398529052734,
  'test/rank_1_of_NP_class/unknown': 0.8975741267204285,
  'test/rank_1_of_NP_class/Coumarins':

In [8]:
# spectra-flexible model
# usually not gonna use it because it tests right away after training 

# load model 

model_path = Path("/root/gurusmart/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes/flexible_384_trail1")
hyperpaerameters_path = model_path / "hparams.yaml"

checkpoint_path = model_path / "checkpoints/epoch=41-step=35994.ckpt"
with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
hparams['use_peak_values'] = False

model = OptionalInputRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)
model.change_ranker_for_testing()

flexible_model_result = trainer.test(model, datamodule)


FileNotFoundError: [Errno 2] No such file or directory: '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes/flexible_384_trail1/hparams.yaml'

In [ ]:

names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
out_file = "temp_model_show.txt"
with open(out_file, "w") as file:
    for name in  names:
        file.write(f"\n{name:10}: ")
        for m in metric_to_focus:
            file.write(f"{m}: {flexible_model_result[0]['test_mean_'+m+'_'+name]:.4f}, ")


In [ ]:
flexible_model_result[0] .keys()    


dict_keys(['test_mean_ce_loss_all_inputs', 'test_mean_pos_loss_all_inputs', 'test_mean_neg_loss_all_inputs', 'test_mean_pos_neg_loss_all_inputs', 'test_mean_cos_all_inputs', 'test_mean_active_bits_all_inputs', 'test_mean_f1_all_inputs', 'test_mean_precision_all_inputs', 'test_mean_recall_all_inputs', 'test_mean_accuracy_all_inputs', 'test_mean_mean_rank_all_inputs', 'test_mean_rank_1_all_inputs', 'test_mean_rank_5_all_inputs', 'test_mean_rank_10_all_inputs', 'test_mean_ce_loss_HSQC_H_NMR', 'test_mean_pos_loss_HSQC_H_NMR', 'test_mean_neg_loss_HSQC_H_NMR', 'test_mean_pos_neg_loss_HSQC_H_NMR', 'test_mean_cos_HSQC_H_NMR', 'test_mean_active_bits_HSQC_H_NMR', 'test_mean_f1_HSQC_H_NMR', 'test_mean_precision_HSQC_H_NMR', 'test_mean_recall_HSQC_H_NMR', 'test_mean_accuracy_HSQC_H_NMR', 'test_mean_mean_rank_HSQC_H_NMR', 'test_mean_rank_1_HSQC_H_NMR', 'test_mean_rank_5_HSQC_H_NMR', 'test_mean_rank_10_HSQC_H_NMR', 'test_mean_ce_loss_HSQC_C_NMR', 'test_mean_pos_loss_HSQC_C_NMR', 'test_mean_neg_loss_